# II - The Reactive Covariance Model - Paper 2
In this section we derive from the leverage a new volatility measure, the reactive volatility introduced by Valeyre and al. in 2013. This measure allows to obtain a more efficient risk measure including the leverage effect and instantaneous price variations. 

In [1]:
# Import relevant libraries

import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy as sp
import numpy as np 
import math
import datetime
import time 

import import_ipynb
import PortfolioOptimization as po
# importer l'Excel de prix 

index_df     = pd.read_excel('Data\indices.xls') 
index_df_out = index_df[len(index_df)-250:len(index_df)] # Out sample (calcul de la performance du portefeuille basé sur la matrice de variance covariance réactiv)
index_df_out = index_df_out.reset_index(drop=True)
index_df     = index_df[0:len(index_df)-250]             # In sample (estimation des matrices de variance covariance réactive)
index_df.head()

ModuleNotFoundError: No module named 'import_ipynb'

### 2.1 - Realized Volatility

One basic approach for dynamically estimated the volatility is to consider the empirical estimator of the volatility over a rolling window:
$$
    \hat{\sigma}_{t,n} = \dfrac{1}{n}\sum\limits_{i=1}^{n}(x_{t-i+1} - \bar{x}_{t,n})^{2})
$$
where $\bar{x}_{t,n} = \dfrac{1}{n}\sum\limits_{i=1}^{n}x_{t-i+1}$ represents the mean over time window $[t-n, t]$.

However such estimator is in pratice very poor since it does not reflect any instantaneous price variation (a strong innovation is mixed equally with minor ones) and the asymmetric relationship between asset returns and future volatility (leverage effect).

In [ ]:
index_dictionnary = {0 : "STOXX 600", 1:"STOXX 50",2 : "S&P 500",3 : "TOPIX", 4: "FTSE", 5:"DAX", 6:"NIKKEI",7:"NASDAQ"}

def computeRealizedVolatility(Ticker):
    
    n = len(index_df[Ticker])
    annualized_vol = []
    
    # Step 1 : compute the returns 
    gross_returns = (index_df[Ticker]/index_df[Ticker].shift(1)).dropna()
    
    # Step 2 : compute the 250-d vol
    for i in range(250,n):
        annualized_vol.append(np.sqrt(np.cov(gross_returns[i-250:i],gross_returns[i-250:i])[0][1]) * np.sqrt(250)) # on commence a calculer a partir row 250
    return annualized_vol

Ticker = "STOXX 600" # changer ici pour avoir la vol réalisée du ticker souhaité 

dates = index_df["Date"][250:len(index_df["Date"])]
dates = [pd.to_datetime(d) for d in dates]  # Conversion des dates dans le bon format
                         
volatility = {"Date" : dates, "Volatility" : computeRealizedVolatility(Ticker)} # Création d'un dictionnaire pour la vol
volatility_df =pd.DataFrame(volatility)

plt.style.use('seaborn')
volatility_df.plot(x= "Date", y='Volatility', figsize=(10, 8), grid=True)
plt.xlabel('Date')
plt.ylabel('Annualized Volatility')
plt.title('Historical 250d Volatility')
plt.show()
    


### 2.2 - Reactive Volatility Model

__*Reactive volatilty in the stock index case*__

Let $I(t)$ be the stock index level at time $t$. Due to heteroskedasticity in asset returns, $\frac{{\Delta}{I(t+1)}}{I(t)}$, partly caused by price-volatility correlation, the authors define an estimator, $L(t)$, of the stock index, $I(t)$, such that the re-normalized arithmetic returns, $\frac{{\Delta}{I(t+1)}}{L(t)}$, become more homoscedastic. The later, is derived from two exponential weighted moving averages: the slow level, $L_s(t)$, and the fast level, $L_f(t)$, characterizing respectively the dynamic of the retarded and panic effect.

\begin{align}
    \textit{Slow level : }& L_s(t+1) = (1-\lambda_{s})L_{s}(t)+\lambda_{s}I(t+1) \label{eq: slow level}
    \\
    \textit{Fast level : }& L_f(t+1) = (1-\lambda_{f})L_{f}(t)+\lambda_{f}I(t+1) \label{eq: fast level}
\end{align}

where $\lambda_s$ and $\lambda_f$ represent decay rates of the leverage effect for specific and systematic risk. In order to make the estimator $L(t)$ more robust to outliers, the following filter function is applied to the slow and fast price level:

\begin{equation}\label{eq: fonction filtre}
    F_{\phi}(z) = \begin{cases}
    \dfrac{tanh(\phi z)}{\phi} & \phi>0\\
    \hfil z & \phi=0
    \end{cases}
\end{equation}

where $\phi$ represents the region of linearity of the function, i.e. the function is proportional to $z$ for small values of $\lvert z\rvert$ and converge toward a constant, $\lvert\frac{1}{\phi}\rvert$, for large values of $\lvert z\rvert$. The use of this filter assumes that the slow and fast price level cannot diverge more than $\lvert\frac{1}{\phi}\rvert$ from the current level of the price index. By restricting the price dynamics we then limit the impact of extreme events on the time series. 

Based on this filter, the new price level $L(t+1)$ at time $t+1$ is defined as the filtered slow level, $\hat{L}_{s}(t+1)$, modulated by the panic effect measure, $\hat{L}_f(t+1)$.

\begin{align}
    \textit{Slow level : } & \hat{L}_{s}(t+1) = I(t+1)\left((1+F_{\phi}\left(\frac{L_{s}(t+1)}{I(t+1)}-1\right)\right) \label{eq: filtered slow level}
    \\
    \textit{Fast level : } & \hat{L}_{f}(t+1)=1+F_{\phi}\left({\left(\frac{L_{f}(t+1)}{I(t+1)}\right)}^{l}-1\right) \label{eq: filtered fast level}
\end{align}
	\\
where $l = 8$ is the leverage parameter corresponding to the amplitude of the leverage effect between stock returns and volatility (Bouchaud and al., 2001). Finally, the new price level $L(t+1)$ is given by:

\begin{equation}\label{eq: nouveaux prix}
    L(t+1) = \hat{L}_{s}(t+1)\left(1+F_{\phi}\left({\left(\frac{L_{f}(t+1)}{I(t+1)}\right)}^{l}-1\right)\right)
\end{equation} 

Derived from the re-normalized returns, $\frac{{\Delta}{I(t+1)}}{L(t)}$, a first approximation of the variance is obtained using an EWMA of parameter $\lambda_{\sigma} \approx \frac{1}{40} (economic uncertainty does not change significantly in a period of two months):

\begin{equation}
    \hat{\sigma}_{I}^{2}(t+1)
    =
    (1-\lambda_{\sigma})
    *
    \hat{\sigma}_{I}^{2}(t)
    +
    \lambda_{\sigma}
    *
    \left(\frac{{\Delta}{I(t+1)}}{L(t)}\right)^{2}
\end{equation}

In order to take into account instantaneous price variations the later is adjusted by the following reactive component $\frac{L(t+1)}{I(t+1)}$:
	
\begin{equation}\label{eq: volatilite reactive}
    \sigma_{I}(t+1)
    =
    \hat{\sigma}_{I}(t+1)
    *
    \frac{L(t+1)}{I(t+1)}
\end{equation}

__*Reactive volatilty in the individual stock case*__

In the case of single stocks, the reactive volatility is obtained by replacing the index level $I(t)$ by the stock price $P_{i}(t)$ in the equation of the slow price level and the filtered slow price level:

\begin{equation}
    \sigma_{i}(t+1)
    =
    \hat{\sigma}_{i}(t+1)
    *
    \frac{L_{i}(t+1)}{P_{i}(t+1)}
\end{equation}

In [ ]:
# Calcul des niveaux du stock index tels que calibrés par Bouchaud & al.

def compute_L_S(Ticker):
    
    lambda_S = 0.0241 # voir comment récupérer cette valeur depuis notre propre calibration
    n = len(index_df[Ticker])
    L_S = np.zeros(n)
    for i in range(1,n):
        L_S[i] = (1-lambda_S)*L_S[i-1] + lambda_S * index_df[Ticker][i]
    return L_S

def compute_L_F(Ticker):
    lambda_F = 0.1484 # voir comment récupérer cette valeur depuis notre propre calibration
    n = len(index_df[Ticker])
    L_F = np.zeros(n)
    for i in range(1,n):
        L_F[i] = (1-lambda_F)*L_F[i-1] + lambda_F * index_df[Ticker][i]
    return L_F

def phi(z):
    phi_parameter = 1/0.3
    return ((z*phi_parameter)/phi_parameter)

def compute_L_S_Hat(Ticker):
    
    n = len(index_df[Ticker])
    L_S = compute_L_S(Ticker)
    L_F = compute_L_F(Ticker)
    L_S_Hat = np.zeros(n)
    
    for i in range(1,n):
        L_S_Hat[i] = index_df[Ticker][i] * (1 + phi((L_S[i] - index_df[Ticker][i])/index_df[Ticker][i]))
    return L_S_Hat    

def compute_L(Ticker):
    
    n = len(index_df[Ticker])
    L_S = compute_L_S(Ticker)
    L_F = compute_L_F(Ticker)
    L_S_Hat = compute_L_S_Hat(Ticker)
    
    L = np.zeros(n)
    l = 8 ## Ce parametre provient du papier
    
    for i in range(1,n):
        L[i] = L_S_Hat[i]*(1+l*(L_F[i] - index_df[Ticker][i])/L_F[i])
    return L

def renormalized_variance(Ticker):
    n = len(index_df[Ticker])
    lambda_sigma = 0.025 ## Ce parametre provient du papier 
    L = compute_L(Ticker)
    
    sigma_tild = np.zeros(n)
    
    for i in range(1,n):
        sigma_tild[i] = (1-lambda_sigma)*sigma_tild[i-1] + lambda_sigma*(((index_df[Ticker][i]-index_df[Ticker][i-1])/L[i])**2)
    return sigma_tild

def reactive_volatility(Ticker):
    
    n = len(index_df[Ticker])
    L_S = compute_L_S(Ticker)
    L_F = compute_L_F(Ticker)
    sigma_tild = renormalized_variance(Ticker)
    sigma_reactive = np.zeros(n)
    l = 8
    
    for i in range(1,n):
        sigma_reactive[i] = sigma_tild[i] *(L_S[i]/index_df[Ticker][i])*(1+l*(L_F[i]-index_df[Ticker][i])/L_F[i])
    return sigma_reactive


In [ ]:
dates = index_df["Date"][250:len(index_df["Date"])]
dates = [pd.to_datetime(d) for d in dates]  # Conversion des dates dans le bon format

Ticker = "STOXX 600"

reactive_vol = np.sqrt(reactive_volatility(Ticker)) * np.sqrt(250)  # Sans oublier d'annualiser + passer a la racine

reactive_vol = reactive_vol[250:len(index_df["Date"])]

react_volatility_dict = {"Date" : dates, "Reactive Volatility" : reactive_vol} # Création d'un dictionnaire pour la vol
react_volatility_df =pd.DataFrame(react_volatility_dict)

plt.style.use('seaborn')


#react_volatility_df.plot(x= "Date", y="Reactive Volatility", figsize=(10, 8), grid=True, color = "darkred")
#react_volatility_df.plot(x= "Date", y='Historical Volatility', figsize=(10, 8), grid=True, color = "slateblue")

x = dates
y = reactive_vol
y2 = computeRealizedVolatility(Ticker)

plt.plot(x, y, color= "darkred")

plt.plot(x, y2, color = "slateblue")

plt.xlabel('Date')
plt.ylabel('Reactive Volatility')
plt.title('Reactive Volatility')

plt.show()

### 2.3 - The Reactive covariance matrix

Based on the reactive volatility framework, one define the reactive covariance as :

\begin{equation}\label{eq: covariance reactive}
    \Sigma_{t+1}^{i,j} = \tilde{\Sigma}^{i,j}_{t+1}\dfrac{L_{t+1}^{i}}{P_{t+1}^{i}}\dfrac{L_{t+1}^{j}}{P_{t+1}^{j}}
\end{equation}
where
\begin{equation*}
    \tilde{\Sigma}_{t+1}^{i,j} = (1-\lambda_{\sigma})\tilde{\Sigma}_{t}^{i,j} + \lambda_{\sigma}(\dfrac{\Delta P_{t}^{i}}{L_{t}^{i}}\dfrac{\Delta P_{t}^{j}}{L_{t}^{j}})
\end{equation*} and $L_{t+1}^{i}$ corresponds to the homoskedastic price level defined in the previous section.

__Remarks:__
    
-  Considering $i=j$ we obtain the reactive volatility.
-  Computing iteratively the previous relation gives :
    
    \begin{equation}\label{eq: covariance reactive developpee}
        \begin{cases}
            \Sigma_{t}^{i,j} = \tilde{\Sigma}_{t}^{i,j}\dfrac{L_{t}^{i}}{P_{t}^{i}}\dfrac{L_{t}^{j}}{P_{t}^{j}}
            \\			
            \tilde{\Sigma}_{t}^{i,j} = \lambda_{\sigma}\sum\limits_{k=1}^{\infty}(1-\lambda_{\sigma})^{k-1}\dfrac{\Delta P_{t-k}^{i}}{L_{t-k}^{i}}\dfrac{\Delta P_{t-k}^{j}}{L_{t-k}^{j}}
        \end{cases}
    \end{equation}
	
    This relation shows two components: (i) a linear one, $\tilde{\Sigma}_{t}^{i,j}$, and (ii) a non-linear, $\frac{L_{t}^{i}}{P_{t}^{i}}\frac{L_{t}^{j}}{P_{t}^{j}}$. The long term dependency between returns is represented by the linear component while the short term relations are represented by the non linear component.

In [ ]:
def compute_L_S(Ticker): # même définition que volatilité réactive
    
    lambda_S = 0.0241 # voir comment récupérer cette valeur depuis notre propre calibration
    n = len(index_df[Ticker])
    index = index_df[Ticker]
    L_S = np.zeros(n)
    for i in range(1,n):
        L_S[i] = (1-lambda_S)*L_S[i-1] + lambda_S * index[i]
    return L_S

def compute_L_F(Ticker): # même définition que volatilité réactive
    lambda_F = 0.1484 # voir comment récupérer cette valeur depuis notre propre calibration
    n = len(index_df[Ticker])
    index = index_df[Ticker]
    L_F = np.zeros(n)
    for i in range(1,n):
        L_F[i] = (1-lambda_F)*L_F[i-1] + lambda_F * index[i]
    return L_F

def phi(z):
    phi_parameter = 1/0.3
    return ((z*phi_parameter)/phi_parameter)

def compute_L_S_Hat(Ticker): # même définition que volatilité réactive
    
    n = len(index_df[Ticker])
    L_S = compute_L_S(Ticker)
    L_F = compute_L_F(Ticker)
    L_S_Hat = np.zeros(n)
    index = index_df[Ticker]
    for i in range(1,n):
        L_S_Hat[i] = index[i] * (1 + phi((L_S[i] - index[i])/index[i]))
    return L_S_Hat    

def compute_L(Ticker): # même définition que volatilité réactive
    
    n = len(index_df[Ticker])
    L_S = compute_L_S(Ticker)
    L_F = compute_L_F(Ticker)
    L_S_Hat = compute_L_S_Hat(Ticker)
    index = index_df[Ticker]
    L = np.zeros(n)
    l = 8 ## Ce parametre provient du papier
    
    for i in range(1,n):
        L[i] = L_S_Hat[i]*(1+l*(L_F[i] - index[i])/L_F[i])
    return L

def renormalized_covariance(Ticker1,Ticker2):
    n = len(index_df[Ticker1])
    lambda_sigma = 0.025 ## Ce parametre provient du papier 
    L_i = compute_L(Ticker1)
    L_j = compute_L(Ticker2)
    index1 = index_df[Ticker1]
    index2 = index_df[Ticker2]
    sigma_tild = np.zeros(n)
    
    for t in range(1,n):
        sigma_tild[t] = (1-lambda_sigma)*sigma_tild[t-1] + lambda_sigma*((index1[t]-index1[t-1])*(index2[t]- index2[t-1])/(L_i[t]*L_j[t]))
    return sigma_tild

def reactive_covariance(Ticker1,Ticker2):
    
    n = len(index_df[Ticker1])
    L_i = compute_L(Ticker1)
    L_j = compute_L(Ticker2)
    index1 = index_df[Ticker1]
    index2 = index_df[Ticker2]
    sigma_tild = renormalized_covariance(Ticker1,Ticker2)
    sigma_reactive = np.zeros(n)
    l = 8
    
    for t in range(1,n):
        sigma_reactive[t] = sigma_tild[t] *(L_i[t]*L_j[t])/(index1[t]*index2[t])
    return sigma_reactive

In [ ]:
start = time.time()

def reactiveCovarianceMatrix(index_dictionnary):
    n = len(index_dictionnary) # Regarde la taille du dictionnaire
    t = len(index_df[index_dictionnary.get(0)]) ## on défini la dimension temporelle par rapport a la longueur du premier set de donnees 
    covariance = np.zeros((n,n,t)) # Création d'une matrice vide de taille n*n*t pour stocker les valeurs de covariances réactives dans le temps
    
    init = 0 # initialisation du j pour le calcul de la matrice de variance covariance 
    for i in range(0,n):
        for j in range(init,n):
            Xi = index_dictionnary.get(i)  # Recupere le nom de la colonne Xi
            Xj = index_dictionnary.get(j)  # Recupere le nom de la colonne Xj
            covariance[i][j] = reactive_covariance(Xi,Xj) ## Calcule la covariance réactive 
            
        if (init >=1):
            k = 1
            for j in range(init-1,0,-1):
                covariance[i][j] = covariance[i-k][j+k]
                k = k + 1
        init = init +1 # on incrémente le point de départ de j pour prendre en compte la symmétrie de la matrice
            
    return covariance

rcov = reactiveCovarianceMatrix(index_dictionnary)
#print(rcov)

end = time.time()  # Pour timer le tout 
print(end - start)


In [ ]:
plt.plot(np.sqrt(252.0*rcov[0,0,250:len(rcov[0,0,:])]))
print(rcov[:,:,len(rcov[0,0,:])-1])

In [ ]:
# Set the achievable portfolio
PortType  = {'MDP':[0.0, 0.0, 0.0], 'ERC':[1.0, 1.0, 0.0], 'GMV':[0.0, 0.0, 1.0]}
# Optimization options (tolerance criterion, maximum iteration)
optoption = [10^-15, 2000]
# Define the reactive covariance matrix
omega = rcov[:,:,len(rcov[0,0,:])-1]

# Equal Risk Contribution Allocation
werc = po.RiskBasedPortfolio(omega, PortType['ERC'], optoption)

# Global Minimum Variance Allocation
wgmv = po.RiskBasedPortfolio(omega, PortType['GMV'], optoption)

#Plot weights allocation
df   = pd.DataFrame({'ERC': 100*werc, 'GMV' : 100*wgmv}, index=range(len(omega[0])))
ax   = df.plot.bar(rot=0)

plt.title("Weights allocation")
plt.xlabel('Asset')
plt.ylabel('Weights (in %)')

In [ ]:
def PortfolioValue(w, index_dictionnary):
    
    nvar = len(index_dictionnary)                  # Recuperer la taille du dictionnaire
    nobs = len(index_df_out[index_dictionnary[0]]) # Recuperer le nombre d'observation
    ptfval = np.zeros((nobs,1))        # Preallocation d'une matrice de zeros de taille t*1 pour stocker la valuer du portefeuille

    ptfval[0,0] = 100
    ptfret = 0.0
    for t in range(1,nobs):
        for i in range(nvar):
            ptfret += ((float(index_df_out[index_dictionnary[i]][t])/float(index_df_out[index_dictionnary[i]][t-1]))-1.0)*w[i]
                               
        ptfval[t,0] = ptfval[t-1,0]*(1+ptfret)
        ptfret = 0.0
    return ptfval

In [ ]:
# Compute portfolio value
ptfercval = PortfolioValue(werc, index_dictionnary)
ptfgmvval = PortfolioValue(wgmv, index_dictionnary)


In [ ]:
# Plot portfolio value (out sample performance)
plt.plot(ptfercval, label = 'ERC')
plt.plot(ptfgmvval, label = 'GMV')
plt.legend(loc='upper left')